In [2]:
from conga2 import *
import numpy as np
import collections as co

from igraph import *
import operator
import itertools
import argparse
import random

In [53]:
teams = 12
players_per_team = 16
min_stayed = 0.7 * players_per_team
transfer_windows = 10

In [44]:
def GenerateFootballGraph(teams, players_per_team, min_stayed, transfer_windows):

    #Generate team_labels
    team_labels = []
    for i in range(0,teams):
        lst = [*range(i*players_per_team, (i+1)*players_per_team)]
        team_labels.append(lst)

    #Generate graph
    test_graph = Graph()
    test_graph.add_vertices(teams*players_per_team)
    for t in range(0,teams):
        for i in range(0,players_per_team):
            for j in range(i,players_per_team-1):
                test_graph.add_edges([(i + t * players_per_team ,j+ 1 + t * players_per_team)])


    #Make transfers
    for times in range(0,transfer_windows):
        for i in range(0,teams-1):
            players_chosen = int((len(team_labels[i]) - min_stayed) / 2)
            for j in range(0,players_chosen):
                player_to_transfer = (team_labels[i][random.randint(0, len(team_labels[i]) - 1)])
                team_labels[i].remove(player_to_transfer)
                new_team = random.randint(0,teams-1)
                if new_team == i:
                    new_team += 1
                    new_team %= teams
                team_labels[new_team].append(player_to_transfer)
                for k in team_labels[new_team]:
                    test_graph.add_edges([(player_to_transfer,k)])
    Graph.simplify(test_graph)
    return test_graph

In [54]:
g = GenerateFootballGraph(teams, players_per_team, min_stayed, transfer_windows)
print(g)

IGRAPH U--- 192 4247 --
+ edges:
  0 --   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  25  43  59
64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  84  87  92  94  95
97  98  99 100 101 102 105 106 108 109 110 111 120 121 122 130 138 151 153 156
165 168
  1 --   0   2   3   4   5   6   7   8   9  10  11  12  13  14  15  17  18  20
22  24  25  27  30  31  33  38  41  46  52  54  63  67  68  72  84 100 119 126
135 138 164
  2 --   0   1   3   4   5   6   7   8   9  10  11  12  13  14  15  17  18  20
21  22  23  25  27  31  32  34  35  36  37  38  41  43  44  45  46  47  52  54
55  61  62  63  64  67  68  70  72  76  78  84  87  93  94  98  99 100 101 102
105 106 111 113 115 119 121 130 138 139 144 145 148 149 150 151 152 153 154
155 157 158 159 168 173
  3 --   0   1   2   4   5   6   7   8   9  10  11  12  13  14  15  24  25  29
30  43  62  65  67  77  79  84  87  94  96  97  98  99 100 101 102 104 105 106
108 109 110 111 115 121 124 129 130 132 133 134 136 137 1

In [51]:
result = conga(g,optimal_count=10)


In [37]:
def GirvanNewman(graph,result):
    while(graph.ecount()>0):
        edge_betweenness = graph.edge_betweenness(False, None, None)
        max_eb = max(edge_betweenness)
        
        edgesToDelete = [graph.es[idx].tuple for idx, eb in enumerate(edge_betweenness) if (eb == max_eb and eb == max_eb)]
        result.append(edgesToDelete)
        graph.delete_edges(edgesToDelete)

In [52]:
res = []
GirvanNewman(g,res)